<a href="https://colab.research.google.com/github/Lizatol/Liza-Tolmacheva-1/blob/main/Class_23_04_Travel_time_(1st_try).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install osmnx mapclassify

In [2]:
!pip install descartes

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
import folium
import mapclassify
import osmnx as ox

In [4]:
murino = ox.geocoder.geocode_to_gdf("Мурино")

In [5]:
murino_polygon = murino.geometry

In [6]:
murino = ox.geocoder.geocode_to_gdf("Мурино, Ленинградская область")
murino_polygon = murino.geometry.iloc[0]

# Граф дорог для пеших и машин
G_walk = ox.graph_from_polygon(murino_polygon, network_type='walk')
G_drive = ox.graph_from_polygon(murino_polygon, network_type='drive')

In [7]:
houses = ox.features.features_from_polygon(murino_polygon, tags={"building": ["apartments", "house", "residential"]})
# Медицинские учреждения
health = ox.features.features_from_polygon(murino_polygon, tags={"amenity": ['hospital', 'clinic', 'doctors', 'health_post']})

houses = houses.to_crs(32636)
health = health.to_crs(32636)

In [8]:
houses = gpd.GeoDataFrame(houses, geometry='geometry')
houses['centroid'] = houses.geometry.centroid
houses.set_geometry('centroid', inplace=True)

In [9]:
houses['geom_centroid'] = houses.geometry.centroid

In [12]:
import geopandas as gpd
houses = gpd.GeoDataFrame(houses, geometry='geometry')
print(houses.geometry.head())

element   id     
relation  6794002    POLYGON ((357183.851 6659584.059, 357175.734 6...
          7555906    POLYGON ((356416.44 6660180.036, 356414.456 66...
          7563809    POLYGON ((356487.897 6660216.311, 356490.724 6...
          7603662    POLYGON ((356368.798 6660044.728, 356382.981 6...
          8017898    POLYGON ((356890.096 6660555.376, 356893.511 6...
Name: geometry, dtype: geometry


In [13]:
houses['geom_centroid'] = houses.geometry.centroid

In [14]:
from shapely.geometry import Point

centroid_types = [type(geom) for geom in houses['geom_centroid']]
from collections import Counter
print(Counter(centroid_types))

Counter({<class 'shapely.geometry.point.Point'>: 272})


In [15]:
houses = houses[[isinstance(geom, Point) for geom in houses['geom_centroid']]].copy()

In [19]:
houses = houses[[isinstance(geom, Point) for geom in houses['geom_centroid']]].copy()

In [20]:
houses.head()

geometry  \
element  id                                                           
relation 6794002  POLYGON ((357183.851 6659584.059, 357175.734 6...   
         7555906  POLYGON ((356416.44 6660180.036, 356414.456 66...   
         7563809  POLYGON ((356487.897 6660216.311, 356490.724 6...   
         7603662  POLYGON ((356368.798 6660044.728, 356382.981 6...   
         8017898  POLYGON ((356890.096 6660555.376, 356893.511 6...   

                 addr:housenumber           addr:street    building  \
element  id                                                           
relation 6794002                4       Охтинская аллея  apartments   
         7555906                6  Воронцовский бульвар  apartments   
         7563809               13    бульвар Менделеева  apartments   
         7603662                4  Воронцовский бульвар  apartments   
         8017898             16/9        улица Шувалова  apartments   

                 building:cladding building:levels design:note  \
element  id                                                      
relation 6794002               NaN             NaN         NaN   
         7555906               NaN              23         NaN   
         7563809               NaN              19         NaN   
         7603662               NaN              23         NaN   
         8017898               NaN              18         NaN   

                 building:year_built design:ref design:year  ...  \
element  id                                                  ...   
relation 6794002                 NaN        NaN         NaN  ...   
         7555906                 NaN        NaN         NaN  ...   
         7563809                 NaN        NaN         NaN  ...   
         7603662                 NaN        NaN         NaN  ...   
         8017898                 NaN        NaN         NaN  ...   

                 payment:debit_cards preschool website source:geometry  \
element  id                                                              
relation 6794002                 NaN       NaN     NaN             NaN   
         7555906                 NaN       NaN     NaN             NaN   
         7563809                 NaN       NaN     NaN             NaN   
         7603662                 NaN       NaN     NaN             NaN   
         8017898                 NaN       NaN     NaN             NaN   

                 check_date survey:date          type opening_date  \
element  id                                                          
relation 6794002        NaN         NaN  multipolygon          NaN   
         7555906        NaN         NaN  multipolygon          NaN   
         7563809        NaN         NaN  multipolygon          NaN   
         7603662        NaN         NaN  multipolygon          NaN   
         8017898        NaN         NaN  multipolygon          NaN   

                                        centroid  \
element  id                                        
relation 6794002   POINT (357193.321 6659585.09)   
         7555906  POINT (356416.845 6660200.404)   
         7563809   POINT (356537.33 6660205.152)   
         7603662  POINT (356384.596 6660059.861)   
         8017898  POINT (356895.473 6660512.594)   

                                   geom_centroid  
element  id                                       
relation 6794002   POINT (357193.321 6659585.09)  
         7555906  POINT (356416.845 6660200.404)  
         7563809   POINT (356537.33 6660205.152)  
         7603662  POINT (356384.596 6660059.861)  
         8017898  POINT (356895.473 6660512.594)  

[5 rows x 50 columns]

In [23]:
import networkx as nx
from shapely.geometry import Point
from tqdm import tqdm
def nearest_node(graph, point):
    if not isinstance(point, Point):
        return None
    return ox.nearest_nodes(graph, point.x, point.y)

def travel_time_to_nearest(graph, point, targets, speed_kph=5):
    source = nearest_node(graph, point)
    times = []
    for target in targets.geometry:
        dest = nearest_node(graph, target)
        try:
            length = nx.shortest_path_length(graph, source=source, target=dest, weight='length')
            time_min = length / (speed_kph * 1000 / 60)  # meters -> minutes
            times.append(time_min)
        except:
            continue
    return min(times) if times else None

# Расчет времени в пути пешком
houses['walk_time_min'] = [
    travel_time_to_nearest(G_walk, pt, health) for pt in tqdm(houses['geom_centroid'])
]

100%|██████████| 272/272 [07:45<00:00,  1.71s/it]


In [25]:
houses[houses['walk_time_min'].isnull()]

,,geometry,addr:housenumber,addr:street,building,building:cladding,building:levels,design:note,building:year_built,design:ref,design:year,...,preschool,website,source:geometry,check_date,survey:date,type,opening_date,centroid,geom_centroid,walk_time_min
element,id,,,,,,,,,,,,,,,,,,,,,
